# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key



## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [16]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bengkulu,ID,-3.8004,102.2655,73.45,91,89,4.50,light rain
1,1,Norman Wells,CA,65.2820,-126.8329,-9.36,77,100,0.00,overcast clouds
2,2,Pitimbu,BR,-7.4706,-34.8086,79.39,76,38,11.50,scattered clouds
3,3,Sao Filipe,CV,14.8961,-24.4956,75.51,64,100,10.49,overcast clouds
4,4,Sobolevo,RU,54.4299,31.8996,10.90,88,100,13.89,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [17]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [18]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) | (city_data_df['Max Temp'] >= min_temp)]

# Display sample data
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bengkulu,ID,-3.8004,102.2655,73.45,91,89,4.50,light rain
1,1,Norman Wells,CA,65.2820,-126.8329,-9.36,77,100,0.00,overcast clouds
2,2,Pitimbu,BR,-7.4706,-34.8086,79.39,76,38,11.50,scattered clouds
3,3,Sao Filipe,CV,14.8961,-24.4956,75.51,64,100,10.49,overcast clouds
4,4,Sobolevo,RU,54.4299,31.8996,10.90,88,100,13.89,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
710,710,Broken Hill,AU,-31.9500,141.4333,72.95,36,0,13.89,clear sky
711,711,Noumea,NC,-22.2763,166.4572,89.58,99,100,1.95,overcast clouds
712,712,Rovenki,UA,48.0833,39.3776,38.55,97,100,15.37,overcast clouds
713,713,Road Town,VG,18.4167,-64.6167,78.84,78,20,11.50,few clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [19]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# YOUR CODE HERE
preferred_cities_df.dropna()
# Display sample data
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bengkulu,ID,-3.8004,102.2655,73.45,91,89,4.50,light rain
1,1,Norman Wells,CA,65.2820,-126.8329,-9.36,77,100,0.00,overcast clouds
2,2,Pitimbu,BR,-7.4706,-34.8086,79.39,76,38,11.50,scattered clouds
3,3,Sao Filipe,CV,14.8961,-24.4956,75.51,64,100,10.49,overcast clouds
4,4,Sobolevo,RU,54.4299,31.8996,10.90,88,100,13.89,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
710,710,Broken Hill,AU,-31.9500,141.4333,72.95,36,0,13.89,clear sky
711,711,Noumea,NC,-22.2763,166.4572,89.58,99,100,1.95,overcast clouds
712,712,Rovenki,UA,48.0833,39.3776,38.55,97,100,15.37,overcast clouds
713,713,Road Town,VG,18.4167,-64.6167,78.84,78,20,11.50,few clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [20]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [21]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bengkulu,ID,73.45,light rain,-3.8004,102.2655,
1,Norman Wells,CA,-9.36,overcast clouds,65.2820,-126.8329,
2,Pitimbu,BR,79.39,scattered clouds,-7.4706,-34.8086,
3,Sao Filipe,CV,75.51,overcast clouds,14.8961,-24.4956,
4,Sobolevo,RU,10.90,overcast clouds,54.4299,31.8996,
...,...,...,...,...,...,...,...
710,Broken Hill,AU,72.95,clear sky,-31.9500,141.4333,
711,Noumea,NC,89.58,overcast clouds,-22.2763,166.4572,
712,Rovenki,UA,38.55,overcast clouds,48.0833,39.3776,
713,Road Town,VG,78.84,few clouds,18.4167,-64.6167,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [22]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [23]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row['Lat'] 
    longitude = row['Lng'] 
    
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params) 
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"] 
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Bengkulu - nearest hotel: Dena Hotel
Norman Wells - nearest hotel: Yamouri Inn
Pitimbu - nearest hotel: Pousada Brisa Mar
Sao Filipe - nearest hotel: Bela Vista
Sobolevo - nearest hotel: No hotel found
Kapaa - nearest hotel: Pono Kai Resort
Sanming - nearest hotel: 三明客家宾馆
East London - nearest hotel: No hotel found
Mahebourg - nearest hotel: Grand Bel Air
Atuona - nearest hotel: Pearl Resort
Carnarvon - nearest hotel: No hotel found
Hobart - nearest hotel: Vibe Hotel Hobart
Rikitea - nearest hotel: Chez Bianca & Benoit
Kapoeta - nearest hotel: Kuleu
Livramento - nearest hotel: No hotel found
Klaksvik - nearest hotel: Hotel Klaksvik
Hermanus - nearest hotel: Aloe guest house
Ca Mau - nearest hotel: Khách sạn Song Ngọc
Punta Arenas - nearest hotel: Hotel Lacolet
Tasiilaq - nearest hotel: Angmagssalik
Trinidad - nearest hotel: Hotel Palace
Talnakh - nearest hotel: Талнах
Busselton - nearest hotel: No hotel found
Lagoa - nearest hotel: Casa da Madrinha
Longyearbyen - 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bengkulu,ID,73.45,light rain,-3.8004,102.2655,Dena Hotel
1,Norman Wells,CA,-9.36,overcast clouds,65.2820,-126.8329,Yamouri Inn
2,Pitimbu,BR,79.39,scattered clouds,-7.4706,-34.8086,Pousada Brisa Mar
3,Sao Filipe,CV,75.51,overcast clouds,14.8961,-24.4956,Bela Vista
4,Sobolevo,RU,10.90,overcast clouds,54.4299,31.8996,No hotel found
...,...,...,...,...,...,...,...
710,Broken Hill,AU,72.95,clear sky,-31.9500,141.4333,Ibis Styles
711,Noumea,NC,89.58,overcast clouds,-22.2763,166.4572,Hotel Gondwana
712,Rovenki,UA,38.55,overcast clouds,48.0833,39.3776,No hotel found
713,Road Town,VG,78.84,few clouds,18.4167,-64.6167,No hotel found


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [24]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == 'No hotel found'].index, inplace = True)
clean_hotel_df = hotel_df.dropna(axis="index", how="any") 
# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bengkulu,ID,73.45,light rain,-3.8004,102.2655,Dena Hotel
1,Norman Wells,CA,-9.36,overcast clouds,65.2820,-126.8329,Yamouri Inn
2,Pitimbu,BR,79.39,scattered clouds,-7.4706,-34.8086,Pousada Brisa Mar
3,Sao Filipe,CV,75.51,overcast clouds,14.8961,-24.4956,Bela Vista
5,Kapaa,US,80.92,clear sky,22.0752,-159.3190,Pono Kai Resort
...,...,...,...,...,...,...,...
706,Wellington,NZ,67.59,broken clouds,-41.2866,174.7756,Travelodge
709,Porto Velho,BR,80.65,scattered clouds,-8.7619,-63.9039,Hotel Slaviero Essential Porto Velho
710,Broken Hill,AU,72.95,clear sky,-31.9500,141.4333,Ibis Styles
711,Noumea,NC,89.58,overcast clouds,-22.2763,166.4572,Hotel Gondwana


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [25]:
# Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [26]:
# Configure the map plot
map_plot_4 = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_4

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)